# Read in Data

In [2]:
import pandas as pd
import numpy as np
from typing import Optional

In [1]:
from custom_datasets import MultiLangDataset, SplitSet
from custom_datasets import ns_dataset
from custom_datasets import Languages
from constants import TokenizerEnum, VectorizerEnum
from custom_vectorizers import get_vectorizer

Repository exists. Updating...
Repository exists. Updating...


In [ ]:
YOR_DATASET: SplitSet = ns_dataset.get(Languages.YORUBA)
HAU_DATASET: SplitSet = ns_dataset.get(Languages.HAUSA)
IBO_DATASET: SplitSet = ns_dataset.get(Languages.IGBO)
PCM_DATASET: SplitSet = ns_dataset.get(Languages.NIGERIAN_PIDGIN)

In [ ]:
# Evaluator
from analysis import compare_results

# Tokenizer
from subword_tokenizer import (
    get_tokenizer,
    wordpiece_tokenize_dataframe,
    get_sentencepiece_tokenizer,
    sentencepiece_tokenize_dataframe,
    get_wordpiece_tokeized_data,
)

# Compare Results
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
def encode_labels(df: pd.DataFrame):
    label_mapping = {"negative": 0, "neutral": 1, "positive": 2}
    df["label_encoded"] = df["label"].str.lower().map(label_mapping)

# Naive Bayes

In [ ]:
def naive_bayes(
    dataset: SplitSet, vectorizer: VectorizerEnum,  tokenizer: Optional[TokenizerEnum] = None
) -> dict:
    """
    Naive Bayes classifier for text classification.
    Args:
        dataset (SplitSet): The dataset to use for training and testing.
        tokenizer (TokenizerEnum): The tokenizer to use.
        If tokenizer is None, then no tokenization is applied. Only vectorization is applied.
        vectorizer (VectorizerEnum): The vectorizer to use.
    Returns:
        dict: A dictionary containing the results of the classification.
        Returns a classification report
    """
    pass

# Logistic Regression

In [ ]:
def logistic_regression(
    dataset: SplitSet, vectorizer: VectorizerEnum,  tokenizer: Optional[TokenizerEnum] = None
) -> dict:
    """
    Logistic Regression classifier for text classification.
    Args:
        dataset (SplitSet): The dataset to use for training and testing.
        tokenizer (TokenizerEnum): The tokenizer to use.
        If tokenizer is None, then no tokenization is applied. Only vectorization is applied.
        vectorizer (VectorizerEnum): The vectorizer to use.
    Returns:
        dict: A dictionary containing the results of the classification.
        Returns a classification report
    """
    pass

# Neural Network

In [ ]:
def neural_network(
    dataset: SplitSet, vectorizer: VectorizerEnum,  tokenizer: Optional[TokenizerEnum] = None
) -> dict:
    """
    Neural Network classifier for text classification.
    Args:
        dataset (SplitSet): The dataset to use for training and testing.
        tokenizer (TokenizerEnum): The tokenizer to use.
        If tokenizer is None, then no tokenization is applied. Only vectorization is applied.
        vectorizer (VectorizerEnum): The vectorizer to use.
    Returns:
        dict: A dictionary containing the results of the classification.
        Returns a classification report
    """
    pass

    df = ns_dataset.get(Languages.HAUSA).train
    # encode_labels(df)

    # df['cleaned_tweet'] = df['tweet'].apply(clean_tweet)

    neural_input = get_wordpiece_tokeized_data(
        df,
        vocab_size=3700,
        tweet_column="cleaned_tweet",
        vectorizer_kwargs={"ngram": (1, 2), "max_features": None},
    )

    model = Sequential()
    model.add(Input(shape=(neural_input.X_train.shape[1],)))

    # Dense layers for TF-IDF input
    # (512, 256, 128)
    # (8, 4, 2)
    model.add(Dense(16))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.4))
    model.add(Dense(16))
    model.add(BatchNormalization())
    model.add(Activation("relu"))
    model.add(Dropout(0.2))
    model.add(Dense(8))
    model.add(BatchNormalization())
    model.add(Dense(3, activation="softmax"))

    model.compile(
        loss="sparse_categorical_crossentropy",
        optimizer=Adam(),
        metrics=["accuracy"],
    )

    X = np.array(neural_input.get_dense_X_train())
    y = np.array(neural_input.y_train)

    model.fit(X, y, epochs=100, batch_size=64, verbose=1)

    # Evaluate the model
    # model.evaluate(neural_input.X_test, neural_input.y_test)
    y_pred = model.predict(
        np.array(neural_input.get_dense_X_test()),
    )
    y_pred_classes = np.argmax(y_pred, axis=1)
    accuracy = accuracy_score(neural_input.y_test, y_pred_classes)
    print(f"Accuracy with filtered tweets {accuracy:.4f}")

    print(
        classification_report(
            neural_input.y_test,
            y_pred_classes,
            target_names=["positive", "neutral", "negative"],
        )
    )

In [ ]:
# result1 = naive_bayes(dataset=YOR_DATASET, vectorizer=VectorizerEnum.TFIDF)
# result2 = naive_bayes(
#     dataset=YOR_DATASET,
#     vectorizer=VectorizerEnum.TFIDF,
#     tokenizer=TokenizerEnum.WORDPIECE,
# )

# compare_results(result1=result1, result2=result2)